In [1]:
import pickle
from matplotlib import pyplot as plt
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, SimpleRNN, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_curve, auc, confusion_matrix
from sklearn.model_selection import train_test_split
from keras import optimizers
from tensorflow import set_random_seed
from numpy.random import seed
%matplotlib inline

Using TensorFlow backend.


In [2]:
#set up matplotlib
params = {'legend.fontsize': 'xx-large',
          'figure.figsize': (30, 15),
         'axes.labelsize': 'xx-large',
         'axes.titlesize':'xx-large',
         'xtick.labelsize':'xx-large',
         'ytick.labelsize':'xx-large'}
plt.rcParams.update(params)

with open('data_with_ratios.pickle', 'rb') as fp:
    data = pickle.load(fp)
data[0]

seed(700)
set_random_seed(700)
# lstm, gru
#100 - 73, 67
#42 - 60, 100
# 200 - 74, 67
# 300 - 70, 16
# 400 - 73, 75 -- 65, 87
# 500 - 69, 12 -- 72, 75
# 600 - 75, 54 -- 76, 37
# 700 - 59, 100

In [3]:
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

new_data = []
bankrupt = []
cols = data[0].keys()
MIN_ROWS = 5
ZSCORE = 1.8
for obj in data:
    if (len(obj.keys()) <= 28):
        continue
    cols = intersection(cols, obj.keys())
cols = sorted(cols)
print(cols)

ground_truth = []
ground_bankrupt = []

for obj in data:
    if not obj['bankrupt']:
        continue
    valid = True
    new_obj = []
    for i in range(MIN_ROWS):
        new_vals = []
        for key in cols:
            if key not in obj:
                valid = False
                break
            if (isinstance(obj[key], list) and key not in ['equity', 'roe', 'bad_solvency', 'debt_equity']):
                if len(obj[key]) < MIN_ROWS:
                    valid = False
                    break
                new_vals.append(obj[key][i]['value'])
        new_obj.append(new_vals)
    if (valid):
        ground_bankrupt.append(True)
        ground_truth.append(new_obj)

for obj in data:
    if obj['bankrupt']:
        continue
    valid = True
    new_obj = []
    for i in range(MIN_ROWS):
        new_vals = []
        for key in cols:
            if key not in obj:
                valid = False
                break
            if (isinstance(obj[key], list) and key not in ['equity', 'roe', 'bad_solvency', 'debt_equity']):
                if len(obj[key]) < MIN_ROWS:
                    valid = False
                    break
                new_vals.append(obj[key][i]['value'])
        new_obj.append(new_vals)
    if (valid):
#         ban = obj['bankrupt']
#         if 'zscore' in obj:
#             for zscore in obj['zscore']:
#                 if zscore['value'] < ZSCORE:
#                     ban = True
        neg_equity = False
        neg_roe = False
        bad_solvency = False
        bad_debt = False
        if 'equity' in obj:
            for equity in obj['equity']:
                if equity['value'] < 0:
                    neg_equity = True
        if 'roe' in obj:
            current_roe = None
            for roe in obj['roe']:
                if current_roe and roe['value'] < current_roe / 2:
                    neg_roe = True
                current_roe = roe['value']
        if 'bad_solvency' in obj:
            for solv in obj['bad_solvency']:
                if solv['value'] < 2:
                    bad_solvency = True
        if 'debt_equity' in obj:
            for de in obj['debt_equity']:
                if de['value'] > 2:
                    bad_debt = True
        bankrupt.append((neg_equity or neg_roe) and (bad_debt or bad_solvency))
        new_data.append(new_obj)

['bankrupt', 'company', 'debt_equity', 'equity', 'solvency', 'ticker', 'x10', 'x15', 'x16', 'x17', 'x2', 'x29', 'x3', 'x41', 'x50', 'x55', 'x7', 'x8']


In [4]:
true_bankrupt = []
zscore_bankrupt = []
for obj in data:
    if obj['bankrupt']:
        true_bankrupt.append(obj['ticker'])
    if ('zscore' in obj):
        for zscore in obj['zscore']:
            if zscore['value'] < 1.8:
                zscore_bankrupt.append(obj['ticker'])
                break
intersect = set(true_bankrupt) & set(zscore_bankrupt)
print('True Bankrupt')
print(len(true_bankrupt))
print(true_bankrupt)
print('Bankrupt by Z score')
print(len(zscore_bankrupt))
print(zscore_bankrupt)
print('Intersect')
print(len(intersect))
print(intersect)

True Bankrupt
44
['NCSO', 'ARO', 'ACI', 'ASNA', 'BONT', 'BKU', 'CPN', 'CIT', 'CMLS', 'CHTR', 'CIE', 'DAL', 'DNKN', 'DYN', 'DEST', 'DDMG', 'EFH', 'FRP', 'CK00007861', 'GGP', 'GGP', 'GM', 'ACF', 'GMAN', 'HCOM', 'ZINCQ', 'LEA', 'LEE', 'LNGG', 'LYB', 'MFRM', 'ODP', 'PIR', 'RFP', 'SHLD', 'SHO', 'SMRT', 'SIX', 'C730', 'CK00015847', '', 'TRCO', 'TSQ', 'TROX']
Bankrupt by Z score
747
['FOX', 'ABT', 'AKR', 'ACCO', 'ARAY', 'ACW', 'ACRX', 'ACET', 'ADOM', 'ADT', 'ADRO', 'AMD', 'AEHR', 'ACY', 'AJRD', 'AES', 'AMG', 'SO', 'AIRI', 'AL', 'AKS', 'ALK', 'ALB', 'ALR', 'ALX', 'ALCO', 'ADS', 'LNT', 'AMOT', 'MDRX', 'ALNY', 'ANAV', 'ALPN', 'AEE', 'ABMC', 'AEP', 'AMSC', 'AMT', 'APU', 'AMP', 'FOLD', 'AMKR', 'AHS', 'APC', 'AVXL', 'AR', 'APA', 'AIV', 'ARCI', 'ARCW', 'ACI', 'PNW', 'AWI', 'ARTX', 'ARQL', 'ARRY', 'ASH', 'ASPN', 'ATRO', 'ASTC', 'T', 'AAWW', 'ATW', 'ADP', 'AVB', 'AVNW', 'AVA', 'ACLS', 'ACHC', 'ACAD', 'ACLZ', 'ADTM', 'ADPT', 'ASNB', 'AGHI', 'AA', 'ALEX', 'ARE', 'Y', 'ATI', 'AGN', 'ALE', 'ALSN', 'ALT', 

In [5]:
not_detected = [b for b in true_bankrupt if b not in zscore_bankrupt]
print(not_detected)
for obj in data:
    if obj['ticker'] in not_detected:
        if 'zscore' in obj:
            print(obj['ticker'])
            print(obj['zscore'])
#         else:
#             print(obj['ticker'])

['NCSO', 'ARO', 'ASNA', 'BONT', 'BKU', 'CIT', 'DEST', 'DDMG', 'CK00007861', 'ACF', 'GMAN', 'ZINCQ', 'LEA', 'LNGG', 'LYB', 'MFRM', 'ODP', 'PIR', 'SMRT', 'CK00015847', '', 'TRCO', 'TROX']
ARO
[{'date': '8/12/10', 'value': 9.42717009992146}, {'date': '3/6/11', 'value': 9.872457180713567}]
ASNA
[{'date': '9/6/11', 'value': 5.83933649983558}, {'date': '1/3/12', 'value': 6.97692340830925}, {'date': '4/3/13', 'value': 4.284513750136817}, {'date': '3/3/14', 'value': 4.790147416052633}, {'date': '3/3/15', 'value': 4.277927708003096}, {'date': '1/3/16', 'value': 2.2732138946707994}, {'date': '8/6/17', 'value': 2.9813499518295137}, {'date': '5/3/18', 'value': 3.031318404692856}]
BONT
[{'date': '7/12/11', 'value': 2.0314942241645726}, {'date': '11/4/12', 'value': 2.2940571223782764}, {'date': '17/4/13', 'value': 2.2748608137511175}, {'date': '16/4/14', 'value': 2.2829921108696043}, {'date': '15/4/15', 'value': 2.2215037249441414}, {'date': '13/4/16', 'value': 2.154344385539776}, {'date': '12/4/17'

In [6]:
arr = np.array(new_data)
test_arr = np.array(ground_truth)
arr.shape

(1362, 5, 13)

In [7]:
def train_model(x_train, y_train, units, dropout, num_lstm_layers, model_type, epoch, batch_size):
    model = Sequential(
        [
            LSTM(units, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])) if model_type == 'lstm' 
            else GRU(units, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])) if model_type == 'gru' 
            else SimpleRNN(units, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])),
            Dropout(dropout)
        ] +
        [
            LSTM(units, return_sequences=True) if model_type == 'lstm' else 
            GRU(units, return_sequences=True) if model_type == 'gru' else
            SimpleRNN(units, return_sequences=True),
            Dropout(dropout)
        ] * (num_lstm_layers - 2) +
        [
            LSTM(units) if model_type == 'lstm' else 
            GRU(units) if model_type == 'gru' else
            SimpleRNN(units),
            Dropout(dropout),
            Dense(1, activation = 'sigmoid')
        ])
    model.compile(optimizer=optimizers.RMSprop(lr=0.003),loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(x_train, y_train, epochs=epoch, batch_size=batch_size)
    return model, history

In [ ]:
scalers = {}
for i in range(arr.shape[2]):
    scalers[i] = MinMaxScaler(feature_range=(0,1))
    arr[:, :, i] = scalers[i].fit_transform(arr[:, :, i])
    test_arr[:, :, i] = scalers[i].transform(test_arr[:, :, i])
x_train, x_test, y_train, y_test = train_test_split(arr, bankrupt, test_size=0.15, random_state=42)
print('bankrupt:')
print(len([True for b in bankrupt if b]))
print('not bankrupt:')
print(len([True for b in bankrupt if not b]))

bankrupt:
445
not bankrupt:
917


In [ ]:
EPOCH = 100
BATCH_SIZE = 30
LSTM_UNITS = 60
DROPOUT = 0.2
NUM_LSTM_LAYERS = 4
lstm_model, lstm_history = train_model(x_train, y_train, LSTM_UNITS, DROPOUT, NUM_LSTM_LAYERS, 'gru', EPOCH, BATCH_SIZE)

Epoch 1/100
1157/1157 [==============================] - 3s 3ms/step - loss: 0.6456 - acc: 0.6638
Epoch 2/100
1157/1157 [==============================] - 1s 677us/step - loss: 0.6363 - acc: 0.6742
Epoch 3/100
1157/1157 [==============================] - 1s 619us/step - loss: 0.6328 - acc: 0.6733
Epoch 4/100
1157/1157 [==============================] - 1s 610us/step - loss: 0.6160 - acc: 0.6638
Epoch 5/100
1157/1157 [==============================] - 1s 633us/step - loss: 0.6126 - acc: 0.6733
Epoch 6/100
1157/1157 [==============================] - 1s 667us/step - loss: 0.5992 - acc: 0.6724
Epoch 7/100
1157/1157 [==============================] - 1s 613us/step - loss: 0.5940 - acc: 0.6776
Epoch 8/100
1157/1157 [==============================] - 1s 619us/step - loss: 0.5993 - acc: 0.6629
Epoch 9/100
1157/1157 [==============================] - 1s 617us/step - loss: 0.5906 - acc: 0.6638
Epoch 10/100
1157/1157 [==============================] - 1s 634us/step - loss: 0.5804 - acc: 0.6646
E

1157/1157 [==============================] - 1s 669us/step - loss: 0.4828 - acc: 0.7537
Epoch 83/100
1157/1157 [==============================] - 1s 652us/step - loss: 0.4887 - acc: 0.7476
Epoch 84/100
1157/1157 [==============================] - 1s 638us/step - loss: 0.4912 - acc: 0.7442
Epoch 85/100
1157/1157 [==============================] - 1s 637us/step - loss: 0.4946 - acc: 0.7476
Epoch 86/100
1157/1157 [==============================] - 1s 648us/step - loss: 0.4851 - acc: 0.7545
Epoch 87/100
1157/1157 [==============================] - 3s 2ms/step - loss: 0.4961 - acc: 0.7511
Epoch 88/100
1157/1157 [==============================] - 1s 637us/step - loss: 0.4855 - acc: 0.7468
Epoch 89/100
1157/1157 [==============================] - 1s 641us/step - loss: 0.4904 - acc: 0.7347
Epoch 90/100
1157/1157 [==============================] - 1s 637us/step - loss: 0.4853 - acc: 0.7597
Epoch 91/100
1157/1157 [==============================] - 1s 655us/step - loss: 0.4812 - acc: 0.7528
Epoch

In [ ]:
def plot_loss(history):
    plt.plot(history.history['loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.show()
    
plot_loss(lstm_history)

In [ ]:
# Evaluate model and predict data
scores = lstm_model.evaluate(x_test, y_test)
print("model: \n%s: %.2f%%" % (lstm_model.metrics_names[1], scores[1]*100))

y_predict = lstm_model.predict_classes(x_test)
cm = confusion_matrix(y_test, y_predict)
print(cm)

fpr_BDmodel, tpr_BDmodel, thresholds_BDmodel = roc_curve(y_test, y_predict)
auc_BDmodel = auc(fpr_BDmodel, tpr_BDmodel)
print(auc_BDmodel)

In [ ]:
tn, fp, fn, tp = cm.ravel()
(tn, fp, fn, tp)

In [ ]:
# Evaluate model and predict data
scores = lstm_model.evaluate(test_arr, ground_bankrupt)
print("model: \n%s: %.2f%%" % (lstm_model.metrics_names[1], scores[1]*100))

y_predict = lstm_model.predict_classes(test_arr)
cm = confusion_matrix(ground_bankrupt, y_predict)
print(cm)

# fpr_BDmodel, tpr_BDmodel, thresholds_BDmodel = roc_curve(ground_bankrupt, y_predict)
# auc_BDmodel = auc(fpr_BDmodel, tpr_BDmodel)
# print(auc_BDmodel)